In [1]:
import json
f = open("./data/data/movie3.json", "r")
movie_data = json.load(f)
f.close()

# project_budget nan값 갯수 확인하기

In [36]:
not_korean_list =[]
for i in range(932):
    if not any(u"한국" in s for s in movie_data[i]['nation']):
         not_korean_list.append(i)

print len(not_korean_list) 

872


In [38]:
count = 0
for i in not_korean_list:
    if movie_data[i]['project_budget'] != 'NA':

        count +=1
print count

622


In [39]:
nan_budget_list = []
for i in not_korean_list:
    if movie_data[i]['project_budget'] == 'NA':
        nan_budget_list.append(movie_data[i]['Mname'])
        
print len(nan_budget_list)

250


# project_budget data 채우기

In [9]:
import pandas as pd
kaggle_data = pd.read_csv('./data/movie_metadata.csv')

In [10]:
kaggle_data.tail()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
5042,Color,Jon Gunn,43.0,90.0,16.0,16.0,Brian Herzlinger,86.0,85222.0,Documentary,...,84.0,English,USA,PG,1100.0,2004.0,23.0,6.6,1.85,456


In [11]:
import re
from collections import OrderedDict

k_budget_dict = OrderedDict()
for i in range(len(kaggle_data['budget'])):
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    k_budget_dict[movie_title] = kaggle_data['budget'][i]

In [13]:
m_name = k_budget_dict.keys()

In [30]:
count =0
for i in range(len(nan_budget_list)):
    if any(nan_budget_list[i].encode('utf-8') in s for s in m_name):
        count +=1
print count

116


In [44]:
import math

In [47]:
count =0
update_dict = {}
for i in range(len(nan_budget_list)):
    if any(nan_budget_list[i].encode('utf-8') in s for s in m_name):
        try:
            if not math.isnan(k_budget_dict[nan_budget_list[i]]):
                #print nan_budget_list[i], k_budget_dict[nan_budget_list[i]]
                update_dict[nan_budget_list[i]] = k_budget_dict[nan_budget_list[i]]
                count +=1
        except:
            pass
print count
print update_dict

94
{u'The Wave': 5000000.0, u'The Cold Light of Day': 20000000.0, u'Lovely, Still': 5000000.0, u'Drive Angry': 50000000.0, u'The Secret in Their Eyes': 2000000.0, u'Point Blank': 3000000.0, u'We Need to Talk About Kevin': 7000000.0, u'Snitch': 15000000.0, u'Legend': 30000000.0, u'Florence Foster Jenkins': 29000000.0, u'Begin Again': 8000000.0, u"God's Not Dead 2": 5000000.0, u'The Hobbit: The Battle of the Five Armies': 250000000.0, u'Quartet': 11000000.0, u'The Hobbit: The Desolation of Smaug': 225000000.0, u'The Assassin': 15000000.0, u"While We're Young": 10000000.0, u'Inside Llewyn Davis': 11000000.0, u'Shame': 6500000.0, u'The Cabin in the Woods': 30000000.0, u'Dragon Blade': 65000000.0, u'Little Boy': 20000000.0, u'A Dangerous Method': 15000000.0, u'Alpha and Omega': 20000000.0, u'The Age of Adaline': 25000000.0, u'The Second Mother': 4000000.0, u'The 33': 26000000.0, u'August: Osage County': 25000000.0, u'Instructions Not Included': 5000000.0, u'The Amazing Spider-Man 2': 200000

# db에 update

In [17]:
from pymongo import MongoClient

In [20]:
mongo = MongoClient(mongo_server, 27017)

In [21]:
movie = mongo.project.movie

In [52]:
m_names = update_dict.keys()

In [59]:
for i in range(len(update_dict)):
    result = movie.update_one({'Mname': m_names[i]}, {'$set' :{'project_budget':update_dict[m_names[i]]}} )
print result.modified_count

1


# actor, director nan개수 확인하기

In [75]:
for i in not_korean_list:
    print movie_data[i]['Mname']
    print movie_data[i]['actor']
    print

Lovelace
{u'Amanda Seyfried': u'/people/chart/?view=Actor&id=amandaseyfried.htm', u'James Franco': u'/people/chart/?view=Actor&id=jamesfranco.htm', u'Juno Temple': u'/people/chart/?view=Actor&id=junotemple.htm', u'Sharon Stone': u'/people/chart/?view=Actor&id=sharonstone.htm', u'Adam Brody': u'/people/chart/?view=Actor&id=adambrody.htm', u'Bobby Cannavale': u'NA', u'Eric Roberts': u'NA', u'Hank Azaria': u'NA', u'Chris Noth': u'NA', u'Wes Bentley': u'NA', u'Chloe Sevigny': u'/people/chart/?view=Actor&id=chloesevigny.htm', u'Robert Patrick': u'NA', u'Peter Sarsgaard': u'/people/chart/?view=Actor&id=petersarsgaard.htm'}

Disconnect
{u'Alexander Skarsg\xe5rd': u'NA', u'Jason Bateman': u'/people/chart/?view=Actor&id=jasonbateman.htm', u'Frank Grillo': u'/people/chart/?view=Actor&id=frankgrillo.htm', u'Alexander Skarsg': u'/people/chart/?view=Actor&id=alexanderskarsgard.htm', u'Paula Patton': u'/people/chart/?view=Actor&id=paulapatton.htm'}

Adore
{u'Naomi Watts': u'/people/chart/?view=Actor

In [442]:
count = 0
nan_actor_list = []
for i in not_korean_list:
    try:
        if movie_data[i]['actor']['People'] == u'/people/?ref=ft':
            nan_actor_list.append(movie_data[i]['Mname'])
            count += 1
    except:
        pass
print count
print len(not_korean_list)
print len(nan_actor_list)

164
872
164


In [40]:
for i in not_korean_list:
    print movie_data[i]['Mname']
    print movie_data[i]['director']
    print

Lovelace
{u'People': u'/people/?ref=ft'}

Disconnect
{u'People': u'/people/?ref=ft'}

Adore
Anne Fontaine

Chronic
{u'People': u'/people/?ref=ft'}

Transformers: Age of Extinction
{u'Michael Bay': u'/people/chart/?view=Director&id=michaelbay.htm'}

The Hobbit: The Battle of the Five Armies
{u'Peter Jackson': u'/people/chart/?view=Director&id=peterjackson.htm'}

Guardians of the Galaxy
{u'James Gunn': u'/people/chart/?view=Director&id=jamesgunn.htm'}

Maleficent
{u'Robert Stromberg': u'/people/chart/?view=Director&id=robertstromberg.htm'}

The Hunger Games: Mockingjay - Part 1
{u'Francis Lawrence': u'/people/chart/?view=Director&id=francislawrence.htm'}

X-Men: Days of Future Past
{u'Bryan Singer': u'/people/chart/?view=Director&id=bryansinger.htm'}

Captain America: The Winter Soldier
{u'Anthony Russo': u'/people/chart/?view=Director&id=anthonyrusso.htm', u'Joe Russo': u'/people/chart/?view=Director&id=joerusso.htm'}

Dawn of the Planet of the Apes
{u'Matt Reeves': u'/people/chart/?vie

In [41]:
count = 0
nan_director_list = []
for i in not_korean_list:
    try:
        if movie_data[i]['director']['People'] == u'/people/?ref=ft':
            nan_director_list.append(movie_data[i]['Mname'])
            count += 1
    except:
        pass
print count
print len(not_korean_list)
print len(nan_director_list)

189
872
189


# director nan값 채우기

In [464]:
for i in not_korean_list:
    print movie_data[i]['director']

{u'People': u'/people/?ref=ft'}
{u'People': u'/people/?ref=ft'}
{u'People': u'/people/?ref=ft'}
{u'People': u'/people/?ref=ft'}
{u'Michael Bay': u'/people/chart/?view=Director&id=michaelbay.htm'}
{u'Peter Jackson': u'/people/chart/?view=Director&id=peterjackson.htm'}
{u'James Gunn': u'/people/chart/?view=Director&id=jamesgunn.htm'}
{u'Robert Stromberg': u'/people/chart/?view=Director&id=robertstromberg.htm'}
{u'Francis Lawrence': u'/people/chart/?view=Director&id=francislawrence.htm'}
{u'Bryan Singer': u'/people/chart/?view=Director&id=bryansinger.htm'}
{u'Anthony Russo': u'/people/chart/?view=Director&id=anthonyrusso.htm', u'Joe Russo': u'/people/chart/?view=Director&id=joerusso.htm'}
{u'Matt Reeves': u'/people/chart/?view=Director&id=mattreeves.htm'}
{u'Marc Webb': u'/people/chart/?view=Director&id=marcwebb.htm'}
{u'Christopher Nolan': u'/people/chart/?view=Director&id=christophernolan.htm'}
{u'Chris Williams': u'/people/chart/?view=Director&id=chriswilliams.htm', u'Don Hall': u'/peo

In [465]:
k_director_dict = {}
for i in range(len(kaggle_data['director_name'])):
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    k_director_dict[movie_title] = kaggle_data['director_name'][i]
print k_director_dict

{'Men with Brooms': 'Paul Gross', 'Vampires Suck': 'Jason Friedberg', 'The Road': 'John Hillcoat', 'Yes Man': 'Peyton Reed', 'Wanderlust': 'David Wain', 'Gremlins': 'Joe Dante', 'Dragon Wars: D-War': 'Hyung-rae Shim', 'Life of Pi': 'Ang Lee', 'Ice Princess': 'Tim Fywell', '[Rec]': 'Jaume Balaguer\xc3\xb3', 'Before Midnight': 'Richard Linklater', 'Kingpin': 'Bobby Farrelly', 'Videodrome': 'David Cronenberg', 'Cradle 2 the Grave': 'Andrzej Bartkowiak', 'The Order': 'Brian Helgeland', 'Red Dog': 'Kriv Stenders', 'Crocodile Dundee II': 'John Cornell', 'The Ice Storm': 'Ang Lee', 'Mulan': 'Tony Bancroft', 'Men of War': 'Perry Lang', 'Nanny McPhee Returns': 'Susanna White', 'Flyboys': 'Tony Bill', 'Eight Legged Freaks': 'Ellory Elkayem', 'The Age of Innocence': 'Martin Scorsese', 'We Need to Talk About Kevin': 'Lynne Ramsay', 'Mongol: The Rise of Genghis Khan': 'Sergey Bodrov', 'Side Effects': 'Steven Soderbergh', 'East Is East': "Damien O'Donnell", 'The Faculty': 'Robert Rodriguez', 'Underd

In [466]:
m_name = k_director_dict.keys()

In [467]:
count = 0
update_dict = {}
for i in range(len(nan_director_list)):
    if any(nan_director_list[i].encode('utf-8') in s for s in m_name):
        try:
            k_director_dict[nan_director_list[i]]
            #print nan_director_list[i], k_director_dict[nan_director_list[i]]
            update_dict[nan_director_list[i]] = k_director_dict[nan_director_list[i]]
            count += 1
        except:
            pass
print count
print len(update_dict)

85
85


In [468]:
movie = mongo.project.movie

In [469]:
m_names = update_dict.keys()

In [470]:
update_dict

{u'10 Cloverfield Lane': 'Dan Trachtenberg',
 u'A Dangerous Method': 'David Cronenberg',
 u'A Separation': 'Asghar Farhadi',
 u'About Time': 'Richard Curtis',
 u'Adore': 'Anne Fontaine',
 u'After.Life': 'Agnieszka Wojtowicz-Vosloo',
 u'Alice Through the Looking Glass': 'James Bobin',
 u'Alpha and Omega': 'Anthony Bell',
 u'Alvin and the Chipmunks: The Road Chip': 'Walt Becker',
 u'Baahubali: The Beginning': 'S.S. Rajamouli',
 u'Begin Again': 'John Carney',
 u'Centurion': 'Neil Marshall',
 u'Cloudy with a Chance of Meatballs 2': 'Cody Cameron',
 u'Daybreakers': 'Michael Spierig',
 u'Deadpool': 'Tim Miller',
 u'Dirty Grandpa': 'Dan Mazer',
 u'Do You Believe?': 'Jon Gunn',
 u'Dragon Blade': 'Daniel Lee',
 u'Eddie the Eagle': 'Dexter Fletcher',
 u"Ender's Game": 'Gavin Hood',
 u'Fifty Shades of Grey': 'Sam Taylor-Johnson',
 u"God's Not Dead 2": 'Harold Cronk',
 u'Hands of Stone': 'Jonathan Jakubowicz',
 u'Hoodwinked Too! Hood vs. Evil': 'Mike Disa',
 u'Hotel Transylvania': 'Genndy Tartakov

In [473]:
for i in range(len(update_dict)):
    result = movie.update_one({'Mname': m_names[i]}, {'$set' :{'director':update_dict[m_names[i]]}} )
print result.modified_count

1


# director 제작영화 list 만들기

In [474]:
directors = k_director_dict.values()

In [475]:
d_names = update_dict.values()

In [476]:
from collections import Counter
directors_dict = Counter(directors)

for d in d_names:
    print d, directors_dict[d]


Anand Tucker 3
Mike Disa 1
Jason Zada 1
Roger Michell 3
Tim Miller 1
Steve Martino 2
Levan Gabriadze 1
David Cronenberg 7
Bruce Beresford 4
Dexter Fletcher 1
Harold Cronk 1
Mabrouk El Mechri 1
Nicholas Fackler 1
Luca Guadagnino 1
Babak Najafi 1
Breck Eisner 3
Thea Sharrock 1
Agnieszka Wojtowicz-Vosloo 1
Florian Henckel von Donnersmarck 2
Neil Marshall 3
Kate Barker-Froyland 1
David Nixon 1
Juan José Campanella 2
Lenny Abrahamson 1
Gilles Paquet-Brenner 1
Genndy Tartakovsky 2
Patricia Riggen 3
Steve Martino 2
Cedric Nicolas-Troyan 1
Gavin Hood 4
Jacques Perrin 2
Giuseppe Tornatore 1
James Marsh 2
Brian Baugh 1
Dennis Gansel 1
Cody Cameron 1
Daniel Lee 2
Patricia Riggen 3
Fatih Akin 1
Genndy Tartakovsky 2
Dan Trachtenberg 1
Lee Toland Krieger 2
Wilson Yip 1
Eugenio Derbez 1
Sam Taylor-Johnson 2
David F. Sandberg 1
Aki Kaurismäki 1
John Carney 2
Breck Eisner 3
Jon Gunn 3
Anna Muylaert 1
Alexandre Aja 4
Jonathan Jakubowicz 1
Phyllida Lloyd 2
Floria Sigismondi 1
Peter Sohn 1
Chan-wook Park 

In [482]:
all_movie_name = k_director_dict.keys()

In [483]:
director_movies_dict = {}
for i in range(len(update_dict)):
    movie_list = []
    for j in range(len(k_director_dict)):
        if update_dict[m_names[i]] == k_director_dict[all_movie_name[j]]:
            movie_list.append(all_movie_name[j])
    director_movies_dict[update_dict[m_names[i]]] = movie_list

In [484]:
director_movies_dict

{'Agnieszka Wojtowicz-Vosloo': ['After.Life'],
 'Aki Kaurism\xc3\xa4ki': ['Le Havre'],
 'Alexandre Aja': ['Mirrors',
  'High Tension',
  'Piranha 3D',
  'The Hills Have Eyes'],
 'Anand Tucker': ['Shopgirl',
  'When Did You Last See Your Father?',
  'Leap Year'],
 'Andrey Konchalovskiy': ['The Nutcracker in 3D', 'Tango & Cash'],
 'Anna Muylaert': ['The Second Mother'],
 'Anne Fontaine': ['Coco Before Chanel', 'Adore'],
 'Anthony Bell': ['Alpha and Omega'],
 'Asghar Farhadi': ['A Separation'],
 'Babak Najafi': ['London Has Fallen'],
 'Breck Eisner': ['The Crazies', 'The Last Witch Hunter', 'Sahara'],
 'Brian Baugh': ['To Save a Life'],
 'Brian Levant': ['The Flintstones',
  'Snow Dogs',
  'Jingle All the Way',
  'Are We There Yet?',
  'The Flintstones in Viva Rock Vegas',
  'The Spy Next Door'],
 'Bruce Beresford': ['Driving Miss Daisy',
  'Double Jeopardy',
  'Mr. Church',
  "Mao's Last Dancer"],
 'Cedric Nicolas-Troyan': ["The Huntsman: Winter's War"],
 'Chan-wook Park': ['Lady Vengean

In [485]:
director = mongo.project.director

In [486]:
for i in range(len(director_movies_dict)):
    value_dict = {}
    for j in range(len(director_movies_dict.values()[i])):
        value_dict[director_movies_dict.values()[i][j]] = 'NA'

    director.insert({'movie': value_dict, 'name': director_movies_dict.keys()[i]}, check_keys=False)

c:\python27\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


# director 제작영화 매출액 구하기

In [487]:
count = 0
movie_gross_list = []

for i in range(len(director_movies_dict)):
    director_gross_dict = {}
    for j in range(len(director_movies_dict.values()[i])):
        for k in not_korean_list:
            if director_movies_dict.values()[i][j] == movie_data[k]['Mname']:
                #print movie_list[i], movie_data[j]['gross_kor']
                
                director_gross_dict[re.sub(r'\.','',director_movies_dict.values()[i][j])] = movie_data[k]['gross_kor']
                count+= 1
    print director_movies_dict.keys()[i], director_gross_dict
    movie_gross_list.append(director_gross_dict)

print count
print len(movie_gross_list)

Genndy Tartakovsky {'Hotel Transylvania': 4702065, 'Hotel Transylvania 2': 9239158}
Mike Disa {'Hoodwinked Too! Hood vs Evil': 1034202}
Peter Ramsey {'Rise of the Guardians': 6491615}
Floria Sigismondi {'The Runaways': 61338}
John Boorman {'Point Blank': 373281}
Sam Taylor-Johnson {'Fifty Shades of Grey': 2483760, 'Nowhere Boy': 75628}
Jonathan Jakubowicz {'Hands of Stone': 8537}
Lenny Abrahamson {'Room': 596351}
Babak Najafi {'London Has Fallen': 5115427}
Anne Fontaine {'Adore': 256744}
Andrey Konchalovskiy {'The Nutcracker in 3D': 222913}
Anna Muylaert {'The Second Mother': 9833}
James Mather {'Lockout': 2834316}
Agnieszka Wojtowicz-Vosloo {'AfterLife': 734938}
Max Joseph {'We Are Your Friends': 254381}
S.S. Rajamouli {'Baahubali: The Beginning': 18963}
Phyllida Lloyd {'The Iron Lady': 382308}
Nicholas Fackler {'Lovely, Still': 105519}
Jacques Perrin {'Oceans': 3747608}
Michael Spierig {'Daybreakers': 923737}
Roger Michell {'Morning Glory': 1432760}
Anthony Bell {'Alpha and Omega': 1

# db에 저장

In [488]:
director = mongo.project.director

In [499]:
for i in range(len(movie_gross_list)):
    for j in range(len(movie_gross_list[i])):
        gross_list = ['NA','NA','NA',movie_gross_list[i].values()[j]]
        result = director.update_one({'name': director_movies_dict.keys()[i]}, {'$set': {'movie.{}'.format(movie_gross_list[i].keys()[j]): gross_list}})

# actor nan값 채우기

In [443]:
for i in not_korean_list:
    print movie_data[i]['actor']

{u'Amanda Seyfried': u'/people/chart/?view=Actor&id=amandaseyfried.htm', u'James Franco': u'/people/chart/?view=Actor&id=jamesfranco.htm', u'Juno Temple': u'/people/chart/?view=Actor&id=junotemple.htm', u'Sharon Stone': u'/people/chart/?view=Actor&id=sharonstone.htm', u'Adam Brody': u'/people/chart/?view=Actor&id=adambrody.htm', u'Bobby Cannavale': u'NA', u'Eric Roberts': u'NA', u'Hank Azaria': u'NA', u'Chris Noth': u'NA', u'Wes Bentley': u'NA', u'Chloe Sevigny': u'/people/chart/?view=Actor&id=chloesevigny.htm', u'Robert Patrick': u'NA', u'Peter Sarsgaard': u'/people/chart/?view=Actor&id=petersarsgaard.htm'}
{u'Alexander Skarsg\xe5rd': u'NA', u'Jason Bateman': u'/people/chart/?view=Actor&id=jasonbateman.htm', u'Frank Grillo': u'/people/chart/?view=Actor&id=frankgrillo.htm', u'Alexander Skarsg': u'/people/chart/?view=Actor&id=alexanderskarsgard.htm', u'Paula Patton': u'/people/chart/?view=Actor&id=paulapatton.htm'}
{u'Naomi Watts': u'/people/chart/?view=Actor&id=naomiwatts.htm', u'Robin

In [446]:
k_actor_dict = {}
for i in range(len(kaggle_data['actor_1_name'])):
    actor_list = []
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    actor_list.append(kaggle_data['actor_1_name'][i])
    actor_list.append(kaggle_data['actor_2_name'][i])
    actor_list.append(kaggle_data['actor_3_name'][i])                 
    k_actor_dict[movie_title] = actor_list
print k_actor_dict

{'Men with Brooms': ['Molly Parker', 'Paul Gross', 'Kari Matchett'], 'Vampires Suck': ['Diedrich Bader', 'Dave Foley', 'Anneliese van der Pol'], 'The Road': ['Viggo Mortensen', 'Charlize Theron', 'Robert Duvall'], 'Yes Man': ['Bradley Cooper', 'Zooey Deschanel', 'Danny Masterson'], 'Wanderlust': ['Justin Theroux', 'Lauren Ambrose', 'Joe Lo Truglio'], 'Gremlins': ['Phoebe Cates', 'Harry Carey Jr.', 'Zach Galligan'], 'Dragon Wars: D-War': ['Robert Forster', 'Aimee Garcia', 'Chris Mulkey'], 'Life of Pi': ['Suraj Sharma', 'Rafe Spall', 'Tabu'], 'Ice Princess': ['Connie Ray', 'Trevor Blumas', 'Amy Stewart'], '[Rec]': ['Manuela Velasco', 'Pablo Rosso', 'Carlos Lasarte'], 'Before Midnight': ['Seamus Davey-Fitzpatrick', 'Ariane Labed', 'Athina Rachel Tsangari'], 'Kingpin': ['Bill Murray', 'Lin Shaye', 'Richard Tyson'], 'Videodrome': ['Debbie Harry', 'Leslie Carlson', 'Reiner Schwarz'], 'Cradle 2 the Grave': ['Jet Li', 'Tom Arnold', 'DMX'], 'The Order': ['Heath Ledger', 'Mark Addy', 'Benno F\xc

In [445]:
print len(k_actor_dict)

4917


In [447]:
count = 0
update_dict = {}
for i in range(len(nan_actor_list)):
    if any(nan_actor_list[i].encode('utf-8') in s for s in k_actor_dict.keys()):
        try:
            k_actor_dict[nan_actor_list[i]]
            #print nan_actor_list[i], k_actor_dict[nan_actor_list[i]]
            update_dict[nan_actor_list[i]] = k_actor_dict[nan_actor_list[i]]
            count += 1
        except:
            pass
print count
print len(update_dict)

37
37


In [461]:
for i in range(len(update_dict)):
    for j in range(3):
        actor = re.sub('\.','', update_dict.values()[i][j])
        result = movie.update_one({'Mname': update_dict.keys()[i]}, {'$set' :{'actor.{}'.format(actor): 'NA'}}, upsert= True )
    

# actor 출연영화 list 만들기

In [ ]:
update_dict.values()

In [ ]:
actor_movies_dict = {}
for i in range(len(update_dict)):
    movie_list = []
    for j in range(len(k_actor_dict)):
        if update_dict[m_names[i]] == k_director_dict[all_movie_name[j]]:
            movie_list.append(all_movie_name[j])
    director_movies_dict[update_dict[m_names[i]]] = movie_list